In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

df_likes = pd.read_sql(
    " SELECT timestamp, user_id, post_id FROM  public.feed_data WHERE  target = 1;",con=engine)

In [2]:
df_likes = df_likes.sort_values('timestamp').reset_index(drop=True)

In [3]:
df_likes

,timestamp,user_id,post_id
0,2021-10-01 06:01:40,129394,1126
1,2021-10-01 06:01:40,94081,5671
2,2021-10-01 06:01:40,59784,307
3,2021-10-01 06:01:40,35892,4003
4,2021-10-01 06:01:40,8663,3837
...,...,...,...
8206340,2021-12-29 23:43:27,97105,5336
8206341,2021-12-29 23:43:27,166379,4105
8206342,2021-12-29 23:43:27,139164,3703
8206343,2021-12-29 23:43:27,38863,6839


In [4]:
df_likes.to_csv('all_likes_df.csv', index=False)

In [5]:
# СКРИПТ СОЗДАНИЯ АЛС ФАКТОРОВ(c DeepSeek)
import numpy as np
import pandas as pd
from scipy import sparse
from implicit.als import AlternatingLeastSquares
import pickle

# 1. Загружаем данные
df_likes = pd.read_csv('all_likes_df.csv')
df_likes['timestamp'] = pd.to_datetime(df_likes['timestamp'])

print(f"Всего лайков: {len(df_likes):,}")
print(f"Всего постов: {df_likes['post_id'].nunique():,}")
print(f"Всего пользователей: {df_likes['user_id'].nunique():,}")

# 2. ТОЛЬКО ВРЕМЕННОЙ СПЛИТ (удалить случайный сплит по ID)
min_time = df_likes['timestamp'].min()
max_time = df_likes['timestamp'].max()
split_time = min_time + (max_time - min_time) * 0.8

print(f"\nSplit время: {split_time}")
print(f"Train период: {min_time} - {split_time}")
print(f"Test период: {split_time} - {max_time}")

# 3. Разделение данных ТОЛЬКО по времени
df_train = df_likes[df_likes['timestamp'] < split_time].copy()
df_test = df_likes[df_likes['timestamp'] >= split_time].copy()

print(f"\nTrain лайков: {len(df_train):,}")
print(f"Test лайков: {len(df_test):,}")

# 4. Исключаем cold-start пользователей из test (тех, кого нет в train)
train_user_set = set(df_train['user_id'].unique())
df_test = df_test[df_test['user_id'].isin(train_user_set)].copy()

# 5. Исключаем cold-start посты из test (тех, кого нет в train)
train_post_set = set(df_train['post_id'].unique())
df_test = df_test[df_test['post_id'].isin(train_post_set)].copy()

print(f"Test лайков после исключения cold-start: {len(df_test):,}")

# 6. Подготовка матрицы для ALS (только на train данных)
train_users_sorted = np.sort(df_train['user_id'].unique())
train_posts_sorted = np.sort(df_train['post_id'].unique())

user_to_idx = {uid: i for i, uid in enumerate(train_users_sorted)}
post_to_idx = {pid: i for i, pid in enumerate(train_posts_sorted)}

rows = [user_to_idx[u] for u in df_train['user_id']]
cols = [post_to_idx[p] for p in df_train['post_id']]

train_matrix = sparse.csr_matrix(
    (np.ones(len(df_train), dtype=np.float32), (rows, cols)),
    shape=(len(user_to_idx), len(post_to_idx))
)

print(f"\nРазмер train матрицы: {train_matrix.shape}")
print(f"Заполненность: {train_matrix.nnz / (train_matrix.shape[0] * train_matrix.shape[1]) * 100:.4f}%")

# 7. Обучаем ALS
print("\nОбучение ALS...")
model = AlternatingLeastSquares(
    factors=20,
    iterations=15,
    regularization=0.01,
    alpha=40,
    random_state=42,
    use_gpu=False
)
model.fit(train_matrix, show_progress=True)

# 8. Нормализуем векторы
def normalize(v):
    norms = np.linalg.norm(v, axis=1, keepdims=True)
    return v / np.clip(norms, 1e-10, None)

user_factors = normalize(model.user_factors)
post_factors = normalize(model.item_factors)

# 9. СОХРАНЯЕМ ФАКТОРЫ ТОЛЬКО ДЛЯ TRAIN ОБЪЕКТОВ (без средних векторов для test!)
# Это ключевое изменение: факторы только для тех, кто был в train
np.save('user_factors_train.npy', user_factors.astype(np.float32))
np.save('post_factors_train.npy', post_factors.astype(np.float32))

with open('als_mappings_train.pkl', 'wb') as f:
    pickle.dump({
        'user_to_idx': user_to_idx,        # Только train пользователи
        'post_to_idx': post_to_idx,        # Только train посты
        'split_time': split_time,          # Время разделения
        'train_users_count': len(train_users_sorted),
        'train_posts_count': len(train_posts_sorted)
    }, f)

df_test.to_csv('test_data_correct.csv', index=False)

# 10. Итог
print(f"\n✅ Обучение завершено БЕЗ УТЕЧЕК!")
print(f"   User factors: {user_factors.shape} (только для {len(train_users_sorted):,} пользователей из train)")
print(f"   Post factors: {post_factors.shape} (только для {len(train_posts_sorted):,} постов из train)")
print(f"   Train лайков: {len(df_train):,}")
print(f"   Test лайков: {len(df_test):,} (только пользователи и посты из train)")
print(f"\n📌 Теперь при создании признаков для LGBM:")
print(f"   - Для объектов из train периода: используем эти факторы")
print(f"   - Для новых объектов (после split_time): факторы = 0 или NaN")
print(f"   - Этот же split_time нужно использовать для LGBM датасета!")

/Users/georgij/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Всего лайков: 8,206,345
Всего постов: 6,831
Всего пользователей: 163,202

Split время: 2021-12-12 00:59:05.600000
Train период: 2021-10-01 06:01:40 - 2021-12-12 00:59:05.600000
Test период: 2021-12-12 00:59:05.600000 - 2021-12-29 23:43:27

Train лайков: 6,326,477
Test лайков: 1,879,868
Test лайков после исключения cold-start: 1,879,751

Размер train матрицы: (163168, 6831)
Заполненность: 0.5638%

Обучение ALS...


/Users/georgij/anaconda3/envs/my_env/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:14<00:00,  4.98s/it]



✅ Обучение завершено БЕЗ УТЕЧЕК!
   User factors: (163168, 20) (только для 163,168 пользователей из train)
   Post factors: (6831, 20) (только для 6,831 постов из train)
   Train лайков: 6,326,477
   Test лайков: 1,879,751 (только пользователи и посты из train)

📌 Теперь при создании признаков для LGBM:
   - Для объектов из train периода: используем эти факторы
   - Для новых объектов (после split_time): факторы = 0 или NaN
   - Этот же split_time нужно использовать для LGBM датасета!


In [6]:
df_likes = pd.read_csv('all_likes_df.csv')
df_likes['timestamp'] = pd.to_datetime(df_likes['timestamp'])

In [7]:
with open('als_mappings_train.pkl', 'rb') as f:
    mappings = pickle.load(f)

print("Ключи в mappings:")
print(list(mappings.keys()))

# Если хотите посмотреть структуру полностью:
print("\nСодержимое mappings:")
for key, value in mappings.items():
    print(f"\n{key}: {type(value)}")
    if isinstance(value, dict):
        print(f"  Длина: {len(value)}")
        print(f"  Пример первых 3 элементов: {list(value.items())[:3]}")
    else:
        print(f"  Значение: {value}")


Ключи в mappings:
['user_to_idx', 'post_to_idx', 'split_time', 'train_users_count', 'train_posts_count']

Содержимое mappings:

user_to_idx: <class 'dict'>
  Длина: 163168
  Пример первых 3 элементов: [(200, 0), (201, 1), (202, 2)]

post_to_idx: <class 'dict'>
  Длина: 6831
  Пример первых 3 элементов: [(1, 0), (2, 1), (3, 2)]

split_time: <class 'pandas._libs.tslibs.timestamps.Timestamp'>
  Значение: 2021-12-12 00:59:05.600000

train_users_count: <class 'int'>
  Значение: 163168

train_posts_count: <class 'int'>
  Значение: 6831


In [8]:
### СКРИПТ ПО СОЗДАНИЮ СЛОВАРЯ ПОСТОВ-КАНДИДАТОВ(c DeepSeek)
import numpy as np
import pandas as pd
from collections import defaultdict
import gc
import pickle
import faiss
from tqdm import tqdm

# Включить многопоточность FAISS
faiss.omp_set_num_threads(8)

# ==================== 1. ОПТИМИЗИРОВАННАЯ ЗАГРУЗКА ====================
print("1. Загрузка данных...")

# Загружаем базу лайков
df_likes = pd.read_csv('all_likes_df.csv')
df_likes['timestamp'] = pd.to_datetime(df_likes['timestamp'])

features_df = pd.read_csv(
    'df_final_clean.csv',
    usecols=['user_id', 'post_id', 'topic_code', 'target'],
    dtype={
        'user_id': 'int32',
        'post_id': 'int32',
        'topic_code': 'int8',
        'target': 'bool'
    }
)

print(f"Features: {len(features_df):,} записей ({features_df.memory_usage().sum()/1024**2:.1f} MB)")

# Загружаем ALS факторы (векторные операции)
post_factors = np.load('post_factors_train.npy').astype('float32')
user_factors = np.load('user_factors_train.npy').astype('float32')

with open('als_mappings_train.pkl', 'rb') as f:
    mappings = pickle.load(f)




user_id_to_idx = mappings['user_to_idx']
post_id_to_idx = mappings['post_to_idx']
# Создаем обратные mapping вручную
user_idx_to_id = {v: k for k, v in user_id_to_idx.items()}
post_idx_to_id = {v: k for k, v in post_id_to_idx.items()}




# Создаем быстрые reverse mapping массивы
user_ids_arr = np.array(list(user_id_to_idx.keys()), dtype=np.int32)
user_indices_arr = np.array(list(user_id_to_idx.values()), dtype=np.int32)

# Создаем словарь post_id -> post_group для быстрого доступа
print("Создаем post_to_group mapping...")
post_to_group_dict = dict(zip(features_df['post_id'], features_df['topic_code']))

# ==================== 2. СУПЕРБЫСТРАЯ ПОДГОТОВКА СТАТИСТИКИ ====================
print("\n2. Подготовка статистики...")

# User-group статистика - СРАЗУ В СЛОВАРЬ (минуя pandas)
print("  User-group stats...")
user_group_dict = {}

# Используем groupby но сразу конвертируем в dict
grouped = features_df.groupby(['user_id', 'topic_code'])
for (user_id, post_group), group in grouped:
    views = len(group)
    likes = group['target'].sum()
    ctr = likes / views if views > 0 else 0.0
    
    user_group_dict[(user_id, post_group)] = {
        'views': int(views),
        'likes': int(likes),
        'ctr': ctr
    }

# Global group статистика - тоже сразу в dict
print("  Global group stats...")
global_group_dict = {}

global_grouped = features_df.groupby('topic_code')['target']
for post_group, targets in global_grouped:
    views = len(targets)
    likes = targets.sum()
    ctr = likes / views if views > 0 else 0.05
    
    global_group_dict[post_group] = {
        'ctr': ctr,
        'ctr_tanh': np.tanh(ctr * 3) * 0.4,
        'views': int(views),
        'likes': int(likes)
    }

# User view counts (для адаптивного лимита)
print("  User view counts...")
user_view_counts_dict = features_df.groupby('user_id').size().to_dict()

# Seen посты (быстрый доступ)
print("  Seen posts...")
user_seen_dict = features_df.groupby('user_id')['post_id'].agg(set).to_dict()

# Liked посты из df_likes (предполагаем, что df_likes уже существует)
print("  Liked posts from df_likes...")
if 'df_likes' in locals() or 'df_likes' in globals():
    user_liked_dict = df_likes.groupby('user_id')['post_id'].agg(set).to_dict()
    print(f"  Loaded {len(user_liked_dict):,} users from df_likes")
else:
    # Fallback: создаем из features_df если df_likes не существует
    print("  WARNING: df_likes not found, using features_df fallback")
    liked_df = features_df[features_df['target'] == True]
    user_liked_dict = liked_df.groupby('user_id')['post_id'].agg(set).to_dict()
    del liked_df

# Освобождаем память
del features_df
gc.collect()

# ==================== 3. СОЗДАНИЕ FAISS ИНДЕКСА ====================
print("\n3. Создание FAISS индекса...")
dimension = post_factors.shape[1]
faiss_index = faiss.IndexFlatIP(dimension)  # Inner Product для cosine similarity
faiss_index.add(post_factors)
print(f"Индекс создан: {faiss_index.ntotal} векторов, размерность {dimension}")

# ==================== 4. УПРОЩЕННАЯ И БЫСТРАЯ ФУНКЦИЯ РАСЧЕТА SCORE ====================

def calculate_novelty_normalized(post_id, seen_set, min_post_id, max_post_id):
    if post_id in seen_set:
        return 0.8  # Было 0.3, стало 0.8 (меньше штраф)
    
    novelty_norm = (post_id - min_post_id) / (max_post_id - min_post_id + 1e-8)
    return 1.0 + novelty_norm * 0.2  # Было 1.0, стало 0.2 (меньше бонус)


def calculate_scores_with_novelty(cosines, post_ids, user_id, seen_set, liked_set):
    """Расчет скора с учетом новизны через post_id."""
    n = len(post_ids)
    scores = cosines.copy()
    
    # Находим мин/макс ID для нормализации
    min_id = min(post_ids)
    max_id = max(post_ids)
    
    for i in range(n):
        post_id = post_ids[i]
        cosine = cosines[i]
        
        # 1. Бонус/штраф за новизну (основной фактор!)
        novelty_bonus = calculate_novelty_normalized(
            post_id, seen_set, min_id, max_id
        )
        scores[i] *= novelty_bonus
        
        # 2. Personal CTR (вторичный фактор)
        post_group = post_to_group_dict.get(post_id, 45)
        key = (user_id, post_group)
        
        if key in user_group_dict:
            stats = user_group_dict[key]
            if stats['views'] >= 3 and stats['ctr'] > 0.1:
                # Аддитивный бонус, ограниченный
                ctr_bonus = min(0.3, stats['ctr'] * 0.7)
                scores[i] += ctr_bonus
        
        # 3. Global CTR (минимальное влияние)
        if post_group in global_group_dict:
            g_bonus = global_group_dict[post_group]['ctr_tanh'] * 0.05
            scores[i] += g_bonus
    
    return scores

# ==================== 5. ОПТИМИЗИРОВАННАЯ БАТЧЕВАЯ ОБРАБОТКА ====================
def process_batch_fast(user_batch, k=300):
    """Супероптимизированная обработка батча."""
    batch_results = {}
    
    # Фильтруем пользователей с факторами
    valid_users = []
    valid_indices = []
    
    for uid in user_batch:
        if uid in user_id_to_idx:
            valid_users.append(uid)
            valid_indices.append(user_id_to_idx[uid])
    
    if not valid_users:
        return {uid: [] for uid in user_batch}
    
    # FAISS поиск для всего батча
    user_vectors = user_factors[valid_indices]
    D_batch, I_batch = faiss_index.search(user_vectors, k=k)
    
    # Обрабатываем каждого пользователя
    for i, user_id in enumerate(valid_users):
        # Получаем кандидатов
        cosines = D_batch[i]
        post_indices = I_batch[i]
        
        # Конвертируем индексы в post_id
        candidate_post_ids = []
        for idx in post_indices:
            if idx < len(post_idx_to_id):
                candidate_post_ids.append(post_idx_to_id[idx])
        
        if not candidate_post_ids:
            batch_results[user_id] = []
            continue
        
        # Фильтруем лайкнутые (быстрая проверка)
        liked_set = user_liked_dict.get(user_id, set())
        seen_set = user_seen_dict.get(user_id, set())
        
        # Фильтрация
        filtered_post_ids = []
        filtered_cosines = []
        
        for post_id, cosine in zip(candidate_post_ids, cosines):
            if post_id not in liked_set:
                # Быстрая пред-фильтрация
                if post_id in seen_set and cosine < 0.6:
                    continue  # пропускаем слаборелевантные просмотренные
                filtered_post_ids.append(post_id)
                filtered_cosines.append(cosine)
        
        if not filtered_post_ids:
            batch_results[user_id] = []
            continue
        
        # Упрощенный расчет скора
        scores = calculate_scores_with_novelty(
            np.array(filtered_cosines, dtype=np.float32),
            filtered_post_ids,
            user_id,
            seen_set,
            liked_set
        )
        
        # Быстрая сортировка топ-N
        top_n = 30
        if len(scores) > top_n:
            # Используем argpartition для быстрого выбора топ-N
            top_indices = np.argpartition(-scores, top_n)[:top_n]
            top_scores = scores[top_indices]
            # Сортируем только топ-N
            sorted_top_indices = top_indices[np.argsort(-top_scores)]
            top_post_ids = [filtered_post_ids[idx] for idx in sorted_top_indices]
        else:
            # Если мало кандидатов, просто сортируем все
            sorted_indices = np.argsort(-scores)
            top_post_ids = [filtered_post_ids[idx] for idx in sorted_indices]
        
        batch_results[user_id] = top_post_ids
    
    # Для пользователей без факторов
    for uid in user_batch:
        if uid not in batch_results:
            batch_results[uid] = []
    
    return batch_results

# ==================== 6. ЗАПУСК БАТЧЕВОЙ ОБРАБОТКИ ====================
print("\n" + "="*60)
print("ЗАПУСК БАТЧЕВОЙ ОБРАБОТКИ")
print("="*60)

# Все пользователи
all_user_ids = np.array(list(user_id_to_idx.keys()), dtype=np.int32)
n_users = len(all_user_ids)
print(f"Всего пользователей для обработки: {n_users:,}")

# Параметры батчинга
batch_size = 10000  # Увеличиваем батч для скорости
all_results = {}

# Оптимальный k для вашего случая (6800 постов)
k_optimal = 300  # 4.4% от всех постов - достаточно!

print(f"Используем k={k_optimal} (FAISS будет искать {k_optimal} ближайших соседей)")
print(f"Размер батча: {batch_size} пользователей")

# Обработка батчами
n_batches = (n_users + batch_size - 1) // batch_size

for batch_idx in tqdm(range(0, n_users, batch_size), total=n_batches, desc="Обработка батчей"):
    batch_end = min(batch_idx + batch_size, n_users)
    user_batch = all_user_ids[batch_idx:batch_end]
    
    batch_results = process_batch_fast(user_batch, k=k_optimal)
    all_results.update(batch_results)
    
    # Освобождаем память каждые 50K пользователей
    if batch_idx % 50000 == 0 and batch_idx > 0:
        gc.collect()

# ==================== 7. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ====================
print("\n7. Сохранение результатов...")

# Сохраняем в pickle
with open('precomputed_candidates_fast_v3.pkl', 'wb') as f:
    pickle.dump(all_results, f)

print(f"✓ Сохранено кандидатов для {len(all_results):,} юзеров")

# Статистика
print("\n" + "="*40)
print("СТАТИСТИКА РЕЗУЛЬТАТОВ")
print("="*40)

result_lengths = [len(posts) for posts in all_results.values()]
counts_30 = sum(1 for x in result_lengths if x == 30)
print(f"Юзеров с 30 кандидатами: {counts_30:,} ({counts_30/n_users*100:.1f}%)")

# Проверка первого юзера
if all_results:
    first_user = list(all_results.keys())[0]
    print(f"\nПример для юзера {first_user}:")
    print(f"  Кандидатов: {len(all_results[first_user])}")
    if len(all_results[first_user]) > 0:
        print(f"  Первые 3: {all_results[first_user][:3]}")

1. Загрузка данных...
Features: 2,316,912 записей (22.1 MB)
Создаем post_to_group mapping...

2. Подготовка статистики...
  User-group stats...
  Global group stats...
  User view counts...
  Seen posts...
  Liked posts from df_likes...
  Loaded 163,202 users from df_likes

3. Создание FAISS индекса...
Индекс создан: 6831 векторов, размерность 20

ЗАПУСК БАТЧЕВОЙ ОБРАБОТКИ
Всего пользователей для обработки: 163,168
Используем k=300 (FAISS будет искать 300 ближайших соседей)
Размер батча: 10000 пользователей


Обработка батчей: 100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [09:24<00:00, 33.21s/it]



7. Сохранение результатов...
✓ Сохранено кандидатов для 163,168 юзеров

СТАТИСТИКА РЕЗУЛЬТАТОВ
Юзеров с 30 кандидатами: 163,168 (100.0%)

Пример для юзера 200:
  Кандидатов: 30
  Первые 3: [4049, 6582, 3503]


In [9]:
# Подготовка DataFrame
sql_data = []
for user_id, post_ids in all_results.items():
    for position, post_id in enumerate(post_ids, 1):  # position от 1 до N
        sql_data.append({
            'user_id': int(user_id),
            'post_id': int(post_id),
            'position': position,  # порядковый номер (1 - лучший)
        })

df_sql_cand = pd.DataFrame(sql_data)

# Сохраняем в CSV для COPY
df_sql_cand.to_csv('recommendations_3.csv', index=False, header=False)

In [10]:
df_sql_cand.head(3)

,user_id,post_id,position
0,200,4049,1
1,200,6582,2
2,200,3503,3


In [11]:
df_sql_cand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4895040 entries, 0 to 4895039
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   post_id   int64
 2   position  int64
dtypes: int64(3)
memory usage: 112.0 MB
